In [1]:
#implementation of the NTRU encryption scheme
#see pp. \S 7.10 and pp. 490-494 Hoffstein, Pipher, Silverman, An Introduction to Mathematical Cryptography
#follow example 7.53 in [HPS]

In [2]:
#define the parameters
N=7; p=3; q=41; d=2
assert N.is_prime()
assert p.is_prime()
assert gcd(N,q) == 1 
assert gcd(p,q) == 1
assert q > (6*d+1)*p

In [3]:
#use conditions f == 1 and g == 0 for homomorphic encryption
homomorphic_encryption = False

In [4]:
#create the polynomial ring \Z[x]/(x^N-1)
P.<x> = PolynomialRing(ZZ)
R = P.quotient(x^N-1); R

Univariate Quotient Polynomial Ring in xbar over Integer Ring with modulus x^7 - 1

In [5]:
#a ternary polynomial in R = \Z[x]/(x^N-1) has d1 coeffs = +1, d2 coeffs = -1
def is_ternary(a,d1,d2):
    assert a in R
    return list(a).count(+1) == d1 and list(a).count(-1) == d2

In [6]:
#create the quotient polynomial ring \Z_p[x]/(x^N-1)
P_p.<x> = PolynomialRing(GF(p))
R_p = P_p.quotient(x^N - 1); R_p

Univariate Quotient Polynomial Ring in xbar over Finite Field of size 3 with modulus x^7 + 2

In [7]:
#create the quotient polynomial ring \Z_q[x]/(x^N-1)
P_q.<x> = PolynomialRing(GF(q))
R_q = P_q.quotient(x^N - 1); R_q

Univariate Quotient Polynomial Ring in xbar over Finite Field of size 41 with modulus x^7 + 40

In [8]:
#define the polynomial f(x). f(x) \in \mathcal{T}(d+1,d), i.e. is a ternary polynomial
#so has d+1 coeffs = 1, d coeffs = -1, all other coeffs = 0
if homomorphic_encryption:
    f = R([1,p,2*p,p,p,p,p])
    assert R_p(f) == 1
else:
    f = R([-1,0,1,1,-1,0,1])
    assert is_ternary(f,d+1,d)
print(f)
print(R_p(f))
print(R_q(f))

xbar^6 - xbar^4 + xbar^3 + xbar^2 - 1
xbar^6 + 2*xbar^4 + xbar^3 + xbar^2 + 2
xbar^6 + 40*xbar^4 + xbar^3 + xbar^2 + 40


In [9]:
#define the polynomial g(x). g(x) \in \mathcal{T}(d,d), i.e. is a ternary polynomial
#so has d coeffs = 1, d coeffs = -1, all other coeffs = 0
if homomorphic_encryption:
    g = R([0,p,p,p,p,p,p])
    assert R_p(g) == 0
else:
    g = R([0,-1,-1,0,1,0,1])
    assert is_ternary(g,d,d)
print(g)
print(R_p(g))
print(R_q(g))

xbar^6 + xbar^4 - xbar^2 - xbar
xbar^6 + xbar^4 + 2*xbar^2 + 2*xbar
xbar^6 + xbar^4 + 40*xbar^2 + 40*xbar


In [10]:
#compute F_p(x) = f(x)^{-1} (mod p)
F_p = R_p(f)^(-1); print(F_p)
assert F_p * R_p(f) == 1

xbar^6 + 2*xbar^5 + xbar^3 + xbar^2 + xbar + 1


In [11]:
#compute F_q(x) = f(x)^{-1} (mod q)
F_q = R_q(f)^(-1); print(F_q)
assert F_q * R_q(f) == 1

8*xbar^6 + 26*xbar^5 + 31*xbar^4 + 21*xbar^3 + 40*xbar^2 + 2*xbar + 37


In [12]:
#define the private key used to decrypt messages
def private_key(f):
    return (f,R_q(f)^(-1))

In [13]:
#define the public key using f(x) (mod p), g(x) (mod p)
def public_key(f,g,q):
    F_q = R_q(f)^(-1)
    return F_q * R_q(g)

In [14]:
print(public_key(f,g,q))

20*xbar^6 + 40*xbar^5 + 2*xbar^4 + 38*xbar^3 + 8*xbar^2 + 26*xbar + 30


In [15]:
#define a plaintext message m(x) with coefficients satisfying -p/2 < m_i <= p/2
#i.e. m is the center lift of a polynomial in R_p
def plaintext(message,p,N):
    assert all([-p/2 < message[i] <= +p/2 for i in range(N)])
    return R(message)

In [16]:
#choose a random element of \mathcal{T}(d,d)
def rand_r(N,d):
    coeffs = (N-2*d)*[0] + d*[+1] + d*[-1]
    sigma = Permutations(N).random_element()
    r = R([coeffs[sigma[i]-1] for i in range(N)])
    assert is_ternary(r,d,d)
    return r

In [17]:
#compute the ciphertext e(x)
def ciphertext(m,f,g,r,p,q,N,d):
    h = public_key(f,g,q)
    e = R_q(p*h*r + m)
    return e

In [18]:
#begin decryption process
def a(f,e):
    return R_q(f*e)

In [19]:
#define the center lift of a from R_q to an element of R = \Z[x]/(x^N-1)
#map coefficients from F_q to Z/qZ (the additive group) and center lift them to \Z
def center_lift(a,modulus):
    return R([Zmod(modulus)(coeff).lift_centered() for coeff in list(a)])

In [20]:
#compute b(x) = F_p(x) * a(x) (mod p) by first center lifting a(x) to R
def b(f,a,q):
    F_p = R_p(f)^(-1)
    return R_p(F_p * center_lift(a,q))

In [21]:
#define the final center_lift for decryption where e = ciphertext, f = secret key, p = prime, q = modulus
def decrypt(e,f,p,q):
    return center_lift(b(f,a(f,e),q),p)

In [22]:
#EXAMPLES

In [23]:
f

xbar^6 - xbar^4 + xbar^3 + xbar^2 - 1

In [24]:
g

xbar^6 + xbar^4 - xbar^2 - xbar

In [25]:
m = plaintext([1,-1,1,1,0,-1,0],p,N); m

-xbar^5 + xbar^3 + xbar^2 - xbar + 1

In [26]:
r = R([-1,1,0,0,0,-1,1]); r

xbar^6 - xbar^5 + xbar - 1

In [27]:
if homomorphic_encryption:
    assert all([abs(coeff) < q/2 for coeff in list(g*r+f*m)])

In [28]:
h = public_key(f,g,q); h

20*xbar^6 + 40*xbar^5 + 2*xbar^4 + 38*xbar^3 + 8*xbar^2 + 26*xbar + 30

In [29]:
e = ciphertext(m,f,g,r,p,q,N,d); e

31*xbar^6 + 19*xbar^5 + 4*xbar^4 + 2*xbar^3 + 40*xbar^2 + 3*xbar + 25

In [30]:
a(f,e)

xbar^6 + 10*xbar^5 + 33*xbar^4 + 40*xbar^3 + 40*xbar^2 + xbar + 40

In [31]:
center_lift(a(f,e),q)

xbar^6 + 10*xbar^5 - 8*xbar^4 - xbar^3 - xbar^2 + xbar - 1

In [32]:
F_p

xbar^6 + 2*xbar^5 + xbar^3 + xbar^2 + xbar + 1

In [33]:
b(f,a(f,e),q)

2*xbar^5 + xbar^3 + xbar^2 + 2*xbar + 1

In [34]:
decrypt(e,f,p,q)

-xbar^5 + xbar^3 + xbar^2 - xbar + 1